In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
inv_data = pd.read_csv('Training_BOP.csv')
inv_data.head()

<ipython-input-4-e4a784c923cc>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  inv_data = pd.read_csv('Training_BOP.csv')


,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1026827,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,No,No,No,Yes,No,No
1,1043384,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.99,0.99,0.0,No,No,No,Yes,No,No
2,1043696,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No
3,1043852,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.10,0.13,0.0,No,No,No,Yes,No,No
4,1044048,8.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-99.00,-99.00,0.0,Yes,No,No,Yes,No,No


In [5]:
# Calculate the total number of rows in the lead_time column
total_rows = inv_data['lead_time'].shape[0]

# Calculate the number of empty (NaN) cells in the lead_time column
empty_cells = inv_data['lead_time'].isna().sum()

# Calculate the percentage of empty cells
percentage_empty = (empty_cells / total_rows) * 100

# Print the result
print(f"Percentage of empty cells in 'lead_time': {percentage_empty:.2f}%")

Percentage of empty cells in 'lead_time': 5.98%


In [6]:
# Remove rows where 'lead_time' is NaN
inv_data_cleaned = inv_data.dropna(subset=['lead_time'])

# Drop the 'sku' column
inv_data_cleaned = inv_data_cleaned.drop(columns=['sku'])

# Optionally, you can reset the index if needed
inv_data_cleaned.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print(inv_data_cleaned)

         national_inv  lead_time  in_transit_qty  forecast_3_month  \
0                 2.0        9.0             0.0               0.0   
1                 7.0        8.0             0.0               0.0   
2                13.0        8.0             0.0               0.0   
3                 6.0        2.0             0.0               0.0   
4                 4.0        8.0             0.0               0.0   
...               ...        ...             ...               ...   
1586962         124.0        8.0           140.0             410.0   
1586963           0.0        2.0             0.0              10.0   
1586964          -1.0        9.0             0.0               7.0   
1586965          62.0        9.0            16.0              39.0   
1586966          19.0        4.0             0.0               0.0   

         forecast_6_month  forecast_9_month  sales_1_month  sales_3_month  \
0                     0.0               0.0            0.0            0.0   
1    

In [7]:
inv_data_cleaned.to_csv('modified_training_bop.csv', index=False)

KeyboardInterrupt: 

Using the modified file hereforth

In [8]:
mod_inv_data = pd.read_csv('modified_training_bop.csv')
mod_inv_data.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.99,0.99,0.0,No,No,No,Yes,No,No
1,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.10,0.13,0.0,No,No,No,Yes,No,No
2,13.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.82,0.87,0.0,No,No,No,Yes,No,No
3,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,Yes,No,Yes,Yes,No,No
4,4.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.82,0.87,0.0,No,No,No,Yes,No,No


In [9]:
mod_inv_data.shape

(349965, 22)

In [10]:
mod_inv_data.describe()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
count,3.499650e+05,349965.000000,349965.000000,3.499650e+05,3.499650e+05,3.499650e+05,349965.000000,3.499650e+05,3.499650e+05,3.499650e+05,349965.000000,349965.000000,349965.000000,349965.000000,349965.000000
mean,4.576183e+02,7.865724,42.446185,1.906193e+02,3.714684e+02,5.433477e+02,56.684411,1.743626e+02,3.433795e+02,5.342098e+02,52.868273,2.187062,-1.083561,-0.619512,0.581152
std,2.513422e+04,7.077886,1059.243965,5.147186e+03,1.022631e+04,1.507274e+04,1910.400869,4.709950e+03,9.232059e+03,1.447862e+04,1046.561662,216.435230,13.535351,11.729343,31.444965
min,-1.349100e+04,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,-99.000000,-99.000000,0.000000
25%,4.000000e+00,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.690000,0.690000,0.000000
50%,1.400000e+01,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000e+00,3.000000e+00,4.000000e+00,0.000000,0.000000,0.840000,0.830000,0.000000
75%,7.900000e+01,9.000000,0.000000,5.000000e+00,1.600000e+01,2.600000e+01,5.000000,1.600000e+01,3.300000e+01,5.100000e+01,4.000000,0.000000,0.970000,0.950000,0.000000
max,1.233440e+07,52.000000,288960.000000,1.218328e+06,2.446072e+06,3.760840e+06,741774.000000,1.076623e+06,2.145715e+06,3.201035e+06,192234.000000,74084.000000,1.000000,1.000000,10024.000000


In [37]:
# Get the value counts for the 'went_on_backorder' column
number_of_values = mod_inv_data['went_on_backorder'].value_counts()

# Print the results
print(number_of_values)

went_on_backorder
No     346496
Yes      3469
Name: count, dtype: int64


There is a severe imbalance between No and Yes Values which is not ideal for a binary classification based model. Therefore, I will sample only some of the 'No' values in order to achieve a 60-40 distribution

In [54]:
# Separate the "yes" and "no" values
yes_values = mod_inv_data[mod_inv_data['went_on_backorder'] == 'Yes']
no_values = mod_inv_data[mod_inv_data['went_on_backorder'] == 'No']

# Randomly sample the "no" values to get 11,305 entries
no_values_sampled = no_values.sample(n=3000, random_state= 1)

# Randomly sample the "yes" values to get 11,305 entries
yes_values_sampled = yes_values.sample(n=3000, random_state= 1)

# Concatenate the "yes" values with the sampled "no" values
modified_data = pd.concat([yes_values_sampled, no_values_sampled])

# Optionally shuffle the resulting DataFrame
modified_data = modified_data.sample(frac=1, random_state= 1).reset_index(drop=True)

eq_data = modified_data['went_on_backorder'].value_counts()

# Check the new distribution
print(modified_data)

      national_inv  lead_time  in_transit_qty  forecast_3_month  \
0             -2.0        8.0             0.0              38.0   
1              1.0        8.0             0.0              10.0   
2              0.0        2.0             0.0               5.0   
3              5.0        2.0             0.0               0.0   
4            685.0        8.0             0.0               0.0   
...            ...        ...             ...               ...   
5995          -1.0        8.0             5.0              70.0   
5996           8.0        8.0             0.0               0.0   
5997         568.0       12.0           164.0             564.0   
5998           8.0        8.0             0.0               0.0   
5999         220.0       12.0             0.0               0.0   

      forecast_6_month  forecast_9_month  sales_1_month  sales_3_month  \
0                 56.0              80.0            8.0           22.0   
1                 10.0              10.0       

In [55]:
# List of columns to replace "Yes" and "No" with 1 and 0
columns_to_replace = ['deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'potential_issue']

# Create a copy of the original DataFrame
super_mod_file = modified_data.copy()

# Replace "Yes" with 1 and "No" with 0 in the specified columns
super_mod_file[columns_to_replace] = super_mod_file[columns_to_replace].replace({'Yes': 1, 'No': 0})

<ipython-input-55-7ebc93aacb91>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  super_mod_file[columns_to_replace] = super_mod_file[columns_to_replace].replace({'Yes': 1, 'No': 0})


In [56]:
super_mod_file.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,-2.0,8.0,0.0,38.0,56.0,80.0,8.0,22.0,36.0,42.0,...,0.0,0.74,0.76,2.0,0,0,0,1,0,Yes
1,1.0,8.0,0.0,10.0,10.0,10.0,3.0,8.0,9.0,9.0,...,0.0,1.00,0.94,0.0,0,0,0,1,0,Yes
2,0.0,2.0,0.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,1,0,0,1,0,Yes
3,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.55,0.77,0.0,0,0,0,1,0,No
4,685.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.00,1.00,0.0,0,0,0,1,0,No


In [57]:
print(super_mod_file.head())  # Display the first few rows
print(super_mod_file.dtypes)   # Display the data types of each column

   national_inv  lead_time  in_transit_qty  forecast_3_month  \
0          -2.0        8.0             0.0              38.0   
1           1.0        8.0             0.0              10.0   
2           0.0        2.0             0.0               5.0   
3           5.0        2.0             0.0               0.0   
4         685.0        8.0             0.0               0.0   

   forecast_6_month  forecast_9_month  sales_1_month  sales_3_month  \
0              56.0              80.0            8.0           22.0   
1              10.0              10.0            3.0            8.0   
2               5.0               5.0            0.0            0.0   
3               0.0               0.0            0.0            0.0   
4               0.0               0.0            0.0            0.0   

   sales_6_month  sales_9_month  ...  pieces_past_due  perf_6_month_avg  \
0           36.0           42.0  ...              0.0              0.74   
1            9.0            9.0  ...  

In [58]:
super_mod_file.groupby('went_on_backorder').mean()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
went_on_backorder,,,,,,,,,,,,,,,,,,,,,
No,359.114000,7.664000,26.550333,140.307000,281.522667,419.748000,45.392000,140.444667,271.367667,414.736,...,0.000333,3.219667,-1.138873,-0.582320,0.160333,0.213667,0.000000,0.119333,0.977000,0.000667
Yes,10.747667,6.372333,4.258333,144.151333,218.062333,292.875667,28.000667,80.247333,142.235000,201.538,...,0.004333,3.835667,-0.483557,-0.024723,4.405667,0.167667,0.000333,0.151667,0.971667,0.000000


In [59]:
import pandas as pd

# Specify the columns to check for NaN values
columns_to_check = [
    'national_inv', 'lead_time', 'in_transit_qty',
    'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
    'sales_1_month', 'sales_3_month', 'sales_6_month',
    'sales_9_month', 'min_bank', 'potential_issue',
    'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg',
    'local_bo_qty', 'deck_risk', 'oe_constraint',
    'ppap_risk', 'stop_auto_buy', 'rev_stop'
]

# Remove rows with NaN values in the specified columns
super_mod_file_cleaned = super_mod_file.dropna(subset=columns_to_check)

X = super_mod_file_cleaned.drop(columns='went_on_backorder')
Y = super_mod_file_cleaned['went_on_backorder']

X = X.dropna()
Y = Y[X.index]

In [60]:
print(X)
print(Y)

      national_inv  lead_time  in_transit_qty  forecast_3_month  \
0             -2.0        8.0             0.0              38.0   
1              1.0        8.0             0.0              10.0   
2              0.0        2.0             0.0               5.0   
3              5.0        2.0             0.0               0.0   
4            685.0        8.0             0.0               0.0   
...            ...        ...             ...               ...   
5995          -1.0        8.0             5.0              70.0   
5996           8.0        8.0             0.0               0.0   
5997         568.0       12.0           164.0             564.0   
5998           8.0        8.0             0.0               0.0   
5999         220.0       12.0             0.0               0.0   

      forecast_6_month  forecast_9_month  sales_1_month  sales_3_month  \
0                 56.0              80.0            8.0           22.0   
1                 10.0              10.0       

In [61]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, stratify=Y, random_state=1)

In [62]:
print(X_train)
print(Y_train)

      national_inv  lead_time  in_transit_qty  forecast_3_month  \
3960          13.0       12.0             0.0               2.0   
4780           6.0        2.0             0.0              40.0   
157           24.0        2.0             0.0               0.0   
315            3.0        8.0             0.0               4.0   
3336        4565.0       12.0             0.0               0.0   
...            ...        ...             ...               ...   
2726        -152.0       12.0             0.0             626.0   
5797           0.0        2.0             0.0               0.0   
2384         374.0       12.0             0.0             320.0   
5831          47.0       12.0             0.0             623.0   
436            7.0        8.0             0.0               0.0   

      forecast_6_month  forecast_9_month  sales_1_month  sales_3_month  \
3960               4.0               6.0            4.0            7.0   
4780              40.0              40.0       

In [63]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
model = LogisticRegression(solver='lbfgs', max_iter=1000)

In [64]:
model.fit(X_train_scaled, Y_train)

LogisticRegression(max_iter=1000)

# **Accuracy on Training Data**

In [65]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.7409259259259259


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [66]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy on test data : ', test_data_accuracy)

Accuracy on test data :  0.7366666666666667


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [96]:
# Original input data
input_data = [-1, 0, 0, 26, 36, 51, 8, 18, 42, 52, 1, 'No', 0, 0, 0, 1, 'No', 'No', 'No', 'No', 'No'
]
# Replace 'No' with 0 and 'Yes' with 1
mod_input_data = [0 if x == 'No' else 1 if x == 'Yes' else x for x in input_data]

mod_input_data_as_numpy_array = np.asarray(mod_input_data)

reshaped_mod_input_data = mod_input_data_as_numpy_array.reshape(1, -1)

predict = model.predict(reshaped_mod_input_data)

print(predict)

if predict == 'Yes':
  print('There will be a shortage')
else:
  print('There will not be a shortage')

['Yes']
There will be a shortage
